Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.639310
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.689787
Minibatch accuracy: 37.5%
Validation accuracy: 47.8%
Minibatch loss at step 100: 1.353418
Minibatch accuracy: 50.0%
Validation accuracy: 61.1%
Minibatch loss at step 150: 0.356923
Minibatch accuracy: 87.5%
Validation accuracy: 74.2%
Minibatch loss at step 200: 0.933966
Minibatch accuracy: 68.8%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.051230
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.565098
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.592616
Minibatch accuracy: 93.8%
Validation accuracy: 78.6%
Minibatch loss at step 400: 0.270172
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.996736
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 500: 0.843099
Minibatch accuracy: 75.0%
Validation accuracy: 81.2%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    maxpool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    conv = tf.nn.conv2d(maxpool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    maxpool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    shape = maxpool.get_shape().as_list()
    reshape = tf.reshape(maxpool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.865362
Minibatch accuracy: 12.5%
Validation accuracy: 12.1%
Minibatch loss at step 50: 1.333262
Minibatch accuracy: 50.0%
Validation accuracy: 50.5%
Minibatch loss at step 100: 1.169738
Minibatch accuracy: 56.2%
Validation accuracy: 65.6%
Minibatch loss at step 150: 0.290796
Minibatch accuracy: 87.5%
Validation accuracy: 75.6%
Minibatch loss at step 200: 1.243008
Minibatch accuracy: 62.5%
Validation accuracy: 78.2%
Minibatch loss at step 250: 0.965808
Minibatch accuracy: 62.5%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.489816
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.673513
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.298199
Minibatch accuracy: 93.8%
Validation accuracy: 81.2%
Minibatch loss at step 450: 0.961759
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.751367
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [7]:
#datasets
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [29]:
# Some constant
batch_size = 10 # smaller to make the CPU ok
patch_size = 3 # still can vary 
depth = 1 # gray scale image
num_labels = 10
image_size = 28
num_channels = 16
hidden_num = 1024

In [86]:
#hypterparameter 
learn_rate = 0.1
beta1 = 0.01
beta2 = 0.01
beta3 = 0.01
beta4 = 0.01
#dropout probability? 


graph = tf.Graph()

with graph.as_default():
    #input
    tf_train_data = tf.placeholder(tf.float32, shape = [batch_size, image_size, image_size, depth])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, num_labels])
    tf_valid_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    #parameters
    with tf.variable_scope('conv1') as scope:
        weight1 = tf.get_variable(
            "conv_weight_1", shape = [patch_size, patch_size, depth, num_channels], initializer=tf.contrib.layers.xavier_initializer())
        biases1 = tf.get_variable("conv_bias_1", initializer = tf.zeros(num_channels))
    
    with tf.variable_scope('conv2') as scope:
        weight2 = tf.get_variable(
            "conv_weight_2", shape = [patch_size, patch_size, num_channels, num_channels], initializer = tf.contrib.layers.xavier_initializer())
        biases2 = tf.get_variable("conv_bias_2", initializer = tf.zeros(num_channels))
    with tf.variable_scope('affine1') as scope:
        weight3 = tf.get_variable("fully_connected_weigths_1", shape = [784, hidden_num], initializer = tf.contrib.layers.xavier_initializer())
        biases3 = tf.get_variable("fully_connected_biases_1", initializer = tf.zeros(hidden_num))
    with tf.variable_scope('affine2') as scope:
        weight4 = tf.get_variable("fully_connected_weights_2", shape = [hidden_num, num_labels], initializer = tf.contrib.layers.xavier_initializer())
        biases4 = tf.get_variable("fully_connected_biases_2", initializer = tf.zeros(num_labels))
    global_step = tf.Variable(0, trainable=False)
    
    #model calculation 
    def model(data, is_training): # is_training for dropout 
        #Conv layer 1
        
        conv1 = tf.nn.conv2d(data, weight1, strides = [1, 1, 1, 1], padding = "SAME") + biases1
        # relu & maxpool
        conv1 = tf.nn.relu(conv1)
        conv1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
        #dropout
        if is_training:
            conv1 = tf.nn.dropout(conv1, 0.5)
        
        #Conv layer 2
        
        conv2 = tf.nn.conv2d(conv1, weight2, strides = [1, 1, 1, 1], padding = "SAME") + biases2
        # relu & maxpool
        conv2 = tf.nn.relu(conv2)
        conv2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")
        #dropout 
        if is_training:
            conv2 = tf.nn.dropout(conv2, 0.5)
        
        #flatten operation 
        shape = conv2.get_shape().as_list()
        flatten = tf.reshape(conv2, shape = [shape[0], shape[1] * shape[2] * shape[3]])

        #affine layer
        affine1 = tf.nn.relu(tf.matmul(flatten, weight3) + biases3)
        #dropout
        if is_training:
            affine1 = tf.nn.dropout(affine1, 0.5)

        logits = tf.matmul(affine1, weight4) + biases4
            
        return logits
        
    #loss
    logits = model(tf_train_data, True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)) + (
        beta1 * tf.nn.l2_loss(weight1) + beta2 * tf.nn.l2_loss(weight2) + beta3 * tf.nn.l2_loss(weight3) + beta4 * tf.nn.l2_loss(weight4))
    
    #optimizers 
    #learning_rate decay
    start_learning_rate = learn_rate
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 500, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step) 
    #prediction & accuracy 
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_data, False))
    test_prediction = tf.nn.softmax(model(tf_test_data, False))


In [87]:
tf.get_default_graph()

In [88]:
graph.collections

[('__varscope',),
 'train_op',
 'trainable_variables',
 ('__variable_store',),
 'variables']

In [89]:
graph.get_collection('trainable_variables')


[<tf.Variable 'conv1/conv_weight_1:0' shape=(3, 3, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv1/conv_bias_1:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv2/conv_weight_2:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 <tf.Variable 'conv2/conv_bias_2:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'affine1/fully_connected_weigths_1:0' shape=(784, 1024) dtype=float32_ref>,
 <tf.Variable 'affine1/fully_connected_biases_1:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'affine2/fully_connected_weights_2:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'affine2/fully_connected_biases_2:0' shape=(10,) dtype=float32_ref>]

In [90]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/predictions.shape[0])

num_iter = 6001 
with tf.Session(graph = graph) as session: 
    #initialization
    session.run(tf.global_variables_initializer())
    print ("initialized")
    #training
    for i in range(num_iter):
        batch_num = (i*batch_size) % (train_dataset.shape[0] - batch_size)
        x_minibatch = train_dataset[batch_num:(batch_num + batch_size), :, :, :]
        y_minibatch = train_labels[batch_num:(batch_num + batch_size), :]
        feed_dict = {tf_train_data:x_minibatch, tf_train_labels:y_minibatch}
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if i % 500 == 0:
            print ("iteration number: {0}, loss: {1}".format(i, l))
            print ("prediction accuracy: {0}%".format(accuracy(prediction, y_minibatch)))
            print ("validation accuracy: {0}%".format(accuracy(valid_prediction.eval(), valid_labels)))
    #testing
    print ("test accuracy: {0}%".format(accuracy(test_prediction.eval(), test_labels)))

initialized
iteration number: 0, loss: 7.05952262878418
prediction accuracy: 10.0%
validation accuracy: 10.09%
iteration number: 500, loss: 3.2952589988708496
prediction accuracy: 80.0%
validation accuracy: 77.16%
iteration number: 1000, loss: 1.664052963256836
prediction accuracy: 80.0%
validation accuracy: 82.19%
iteration number: 1500, loss: 1.359892725944519
prediction accuracy: 80.0%
validation accuracy: 81.55%
iteration number: 2000, loss: 2.0611658096313477
prediction accuracy: 60.0%
validation accuracy: 80.95%
iteration number: 2500, loss: 1.6394010782241821
prediction accuracy: 70.0%
validation accuracy: 82.46%
iteration number: 3000, loss: 1.50675368309021
prediction accuracy: 60.0%
validation accuracy: 81.01%
iteration number: 3500, loss: 1.6431821584701538
prediction accuracy: 80.0%
validation accuracy: 82.96%
iteration number: 4000, loss: 1.2965352535247803
prediction accuracy: 80.0%
validation accuracy: 83.08%
iteration number: 4500, loss: 1.4099926948547363
prediction ac